# Import Modules

In [1]:
import os
import glob
import sys
import json
import pandas as pd
import numpy as np
#import pyperclip
import matplotlib.pyplot as plt
from datetime import datetime
import time

In [ ]:
#for Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/GRPM db/GRPM proteincodinggenes')

# Define gene list - survey directory - ref-mesh list

In [2]:
#Check avalable refs:
folder_path = "ref-mesh-archive"  # Replace with the actual folder path

# Create a file path pattern to match CSV files
file_pattern = os.path.join(folder_path, "*.csv")

# Use glob to get a list of file paths matching the pattern
csv_files = glob.glob(file_pattern)
csv_files_name = []
# Print the list of CSV files
for file in csv_files:
    file_name = os.path.basename(file)
    csv_files_name.append(file_name)

pd.set_option('display.max_rows', 100)
print('Available reference mesh lists:')
csv_files_df = pd.Series(csv_files_name)
print(csv_files_df[csv_files_df.str.contains('ref_mesh_')].reset_index(drop=True))


Available reference mesh lists:
0                       ref_mesh_nest.csv
1              new_ref_mesh_corrected.csv
2                  ref_mesh_neuro_sty.csv
3                      ref_mesh_repro.csv
4              ref_mesh_neuro (no db).csv
5              ref_mesh_nutri (no db).csv
6                     ref_mesh_infect.csv
7                 ref_mesh_neuro_grpm.csv
8                      ref_mesh_nutri.csv
9                ref_mesh_nutri_redux.csv
10          ref_mesh_nutri_grpm_redux.csv
11        ref_mesh_nutri_oldref_redux.csv
12                     ref_mesh_neuro.csv
13                   ref_mesh_dmt2_ms.csv
14               ref_mesh_repro_clean.csv
15                    ref_mesh_ob_bmi.csv
16                       ref_mesh_cvd.csv
17                     ref_mesh_vitam.csv
18                  ref_mesh_cvd_lipo.csv
19                 ref_mesh_eat_taste.csv
20           ref_mesh_eat_taste_clean.csv
21                     ref_mesh_intol.csv
22                     ref_mesh_aller.csv
23

In [3]:
redo= ['xeno',
       'oxi_stress',
       'intol',
       'eat_taste',
       'cvd',
       'dmt2_ms',
       'ob_bmi_less'
       ]
pd.DataFrame(redo).sort_values(by=0)

,0
4,cvd
5,dmt2_ms
3,eat_taste
2,intol
6,ob_bmi_less
1,oxi_stress
0,xeno


In [3]:
#define_directory-------------------------------------------------------
topic_tag = 'ob_bmi'
add = ''
db_tag = 'pcg'
# 'pcg'    = protein coding genes = grpm_db
# 'rna'    = rna genes            = grpm_db_rna
# 'pseudo' = pseudogenes          = grpm_db_pseudo
directory = 'grpm_survey_'+db_tag+'_'+topic_tag+add
save_plot = False

# Import Mesh-reference list
ref = pd.read_csv("ref-mesh-archive/ref_mesh_"+topic_tag+".csv",index_col=0)
if 'mesh' in ref.columns:
    pass
else:
    ref = ref.rename(columns={'Preferred Label': 'mesh'})
ref_mesh_n = ref.mesh.nunique()
ref_mesh_list = ref['mesh'].drop_duplicates()

# create directory and load/create master dataframes-------------------
if not os.path.exists(directory):
    os.makedirs(directory)

if os.path.isfile(directory+'/grpmx_filtered_output.csv'):
    complete_df = pd.read_csv(directory+'/grpmx_filtered_output.csv',index_col=0)
else:
    complete_df = pd.DataFrame()

if os.path.isfile(directory+'/GRPMX_report.csv'):
    df_report_complete = pd.read_csv(directory+'/GRPMX_report.csv',index_col=0)
    restart = True
else:
    df_report_complete = pd.DataFrame()
    restart = False
#----------------------------------------------------------------------
ref
ref_mesh_list

0             Physical Endurance
1              Nutrition Therapy
5               Nutrition Policy
6                 Motor Activity
7          Adipose Tissue, Brown
                 ...            
623    Body Weights and Measures
627                  Body Weight
628      Body Weight Maintenance
629                   Overweight
630                      Obesity
Name: mesh, Length: 243, dtype: object

In [4]:
df_report_complete.T

""


# Load GRPM data from database

In [5]:
#Load GRPM db Report-----------------------------------------
db_name = 'grpm_db_'+db_tag

#get gene list from grpm report
GRPM_report = pd.read_csv(db_name+'/GRPM_report.csv',index_col=0).transpose().reset_index().rename(columns={'index':'gene'})
grpm_genes_list = GRPM_report.gene.to_list()

#Import grpm data back-------------------------------------------
time_load_1 = datetime.now()

columns = ['gene', 'rsid', 'pmids', 'mesh']
dummy = pd.read_csv(db_name+'/grpm_table_output.csv', usecols=columns)

dummy['pmids'] = dummy['pmids'].astype(str) #convert pmid type in str
time_load_2 = datetime.now()
print(time_load_2-time_load_1)

0:00:18.232230


# Comparison module

In [6]:
time_start = datetime.now()

if restart == True:
    restart_from = len(df_report_complete.T)
    gene_start = restart_from
    print('search restarted from '+str(restart_from))
else:
    gene_start = 0

# define gene list
genes = grpm_genes_list[gene_start:]

for gene in genes:

    time_alpha = datetime.now()
    timestamp = time_alpha.strftime('%Y%m%d%H%M%S')

    if gene in dummy.gene.drop_duplicates().to_list():

        dummy_gene = dummy.loc[dummy['gene'] == gene]
        rsidpmid = dummy_gene[['rsid','pmids']].drop_duplicates().reset_index(drop=True)
        pmidmesh = dummy_gene[['pmids','mesh']].drop_duplicates()
        #dfmesh = dummy_gene[['pmids', 'mesh', 'qualifier', 'major']].drop_duplicates().reset_index(drop=True)

        #Filter pmid for rsid with pmid>1
     #   rsidpmid_count = rsidpmid.groupby('rsid').describe().reset_index()
     #   rsidpmid_count.columns = rsidpmid_count.columns.to_flat_index()
     #   new_column_names = ['rsid', 'pmid_count', 'pmid_unique','pmid_top','pmid_freq']
     #   rsidpmid_count.columns = new_column_names
     #   outless = rsidpmid_count[rsidpmid_count.pmid_unique>1]
     #   mask = rsidpmid['rsid'].isin(outless.rsid)
     #   rsidpmid_less = rsidpmid[mask]
     #   #repo parametes
        lit1_rsid = dummy_gene.rsid.nunique()
     #   lit1_pmid_f = rsidpmid_less.pmids.nunique()

        # Correlazione su "pmidmesh.mesh"---------------------------------------
        mask = pmidmesh['mesh'].isin(ref_mesh_list)
        dfmatch = pmidmesh[mask]
        mask_full = dummy_gene['mesh'].isin(ref_mesh_list)
        dfmatch_full = dummy_gene[mask_full]
        #Statistics:
        pmidmesh_before =  pmidmesh.nunique()
        pmidmesh_after =    dfmatch.nunique()
        interesting_pmid =  dfmatch.nunique()

        #pmidmask = rsidpmid['pmids'].isin(dfmatch.pmids) #mymask
        #rsidlast = rsidpmid[pmidmask]  # mask on rsidpmid
        #rsidlastlist = rsidlast.rsid.drop_duplicates()

        #report parameters
        matching_rsid = dfmatch_full['rsid'].nunique()
        dropped_rsid = lit1_rsid - dfmatch_full['rsid'].nunique()

        starting_pmid = pmidmesh['pmids'].nunique()
        starting_mesh = pmidmesh['mesh'].nunique()
        starting_pmidmesh = len(pmidmesh)
        matching_pmids = dfmatch.pmids.nunique()
        matching_mesh = dfmatch.mesh.nunique()
        matching_pmidmesh = len(dfmatch)

        #dfmatch = pmidmesh[mask]
      #  mask = pmidmesh['mesh'].isin(ref['mesh'])
      #  pmidmeshint = pmidmesh.assign(interesting=mask)

        #Analyze enrichment with groupby.describe method--------------------
        dfmatch_less_ = dfmatch_full[['pmids', 'rsid', 'mesh']].drop_duplicates()
      #  interesting_rsid = dfmatch_less_.rsid.nunique()

        #dfmatch_less_['pmids'] = dfmatch_less_['pmids'].astype(str)

        ### groupby.describe analysis by rsid
        dfmatch_less_rsid = dfmatch_less_.groupby('rsid').describe().reset_index()
        dfmatch_less_rsid.columns = dfmatch_less_rsid.columns.to_flat_index()
        new_column_names = ['rsid', 'pmid-count', 'pmid-unique','pmid-top','pmid-freq','mesh-count', 'mesh-unique','mesh-top','mesh-freq']
        dfmatch_less_rsid.columns = new_column_names

        #statistics:
        matching_rsid_pmid10 = len(dfmatch_less_rsid[dfmatch_less_rsid['pmid-unique']>10])
        matching_rsid_pmid100 = len(dfmatch_less_rsid[dfmatch_less_rsid['pmid-unique']>100])

        #sortng, top10
        dfmatch_less_rsidless = dfmatch_less_rsid[['rsid','pmid-unique','mesh-unique']]
        dfmatch_less_rsidlesssort = dfmatch_less_rsidless.sort_values(by='pmid-unique', ascending= False).reset_index(drop=True)
        top10rsid = dfmatch_less_rsidlesssort['rsid'][:10].tolist()

        ### groupby.describe analysis by mesh
        dfmatch_less_mesh = dfmatch_less_.groupby('mesh').describe().reset_index()
        dfmatch_less_mesh.columns = dfmatch_less_mesh.columns.to_flat_index()
        #to handle generate df.groupby.describe, convert Multicolumn to single column
        #https://datascientyst.com/flatten-multiindex-in-pandas/
        new_column_names = ['mesh', 'pmid-count', 'pmid-unique','pmid-top','pmid-freq','rsid-count', 'rsid-unique','rsid-top','rsid-freq']
        dfmatch_less_mesh.columns = new_column_names

        dfmatch_less_mesh_less = dfmatch_less_mesh[['mesh','pmid-unique','rsid-unique']]
        #dfmatch_less_mesh_lesssort = dfmatch_less_mesh_less.sort_values(by='pmid-unique',ascending=False).reset_index(drop=True)

        # add frequency
        samplepmid_count = len(dfmatch.pmids.drop_duplicates())
        dfmatch_less_mesh_less_frq = dfmatch_less_mesh_less.copy()
        mesh_frq = dfmatch_less_mesh_less_frq.loc[:,'pmid-unique'].astype(float)/samplepmid_count
        dfmatch_less_mesh_less_frq.loc[:,'mesh frequency'] = round(mesh_frq,3)#*100
        dfmatch_less_mesh_less_frqsort = dfmatch_less_mesh_less_frq.sort_values(by='pmid-unique',ascending=False).reset_index(drop=True)

        top10mesh = dfmatch_less_mesh_less_frqsort['mesh'][:10].tolist()
        #^display(dfmatch_less_mesh_less_frqsort.head(20))

        if save_plot == True:
            # create a scatter plot
            x = dfmatch_less_mesh_less_frqsort['mesh'].head(30)
            y = dfmatch_less_mesh_less_frqsort['pmid-unique'].head(30)
            plt.figure(figsize=(5, 8))
            plt.title('Scatter Plot: '+gene+' pmid-mesh (filtered)', loc='center',pad=10)
            plt.scatter(y, x)
            plt.gca().invert_yaxis()
            #plt.subplots_adjust(left=0.3, right=0.9, bottom=0.3, top=0.9)
            #plt.xticks(rotation=90)
            plt.tick_params(axis='x', which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
            plt.xlabel('pmid count', position=(0.5, 1.08))
            ax = plt.gca()
            ax.xaxis.set_label_position('top')
            #plt.show()
            plt.savefig(directory+'/'+gene+'_mesh_plot_'+timestamp+'_filtered.png',dpi=120, bbox_inches = "tight")
            plt.close()
        else:
            pass

        #STORE DATA----------------------------------------------------------------------
        #timestamp = time2.strftime('%Y%m%d%H%M%S')

        #comparison results:
        dfmatch_less_['gene'] = gene
        complete_df = pd.concat([complete_df, dfmatch_less_])


        #REPORT------------------------------------------------------------------

        report = { 'reference_mesh': ref_mesh_n,
                   #'filtered_pmidmesh': pmidmesh_after,
                   #'interesting_pmid': interesting_pmid,
                   #'interesting_rsid': interesting_rsid,
                   'starting_pmidmesh': starting_pmidmesh,
                   'starting_pmid' : starting_pmid,
                   'starting_mesh': starting_mesh,
                   'starting_rsid': lit1_rsid,
                   'matching_pmidmesh': matching_pmidmesh,
                   'matching_pmids': matching_pmids,
                   'matching_mesh': matching_mesh,
                   'matching_rsid': matching_rsid,
                   'dropped_rsid': dropped_rsid,
                   'matching_mesh_ratio': round((matching_mesh/starting_mesh),3),
                   'matching_pmids_ratio': round((matching_pmids/starting_pmid),3),
                   'matching_pmidmesh_ratio': round((matching_pmidmesh/starting_pmidmesh),3),
                   'matching_rsid_ratio': round((matching_rsid/lit1_rsid),3),
                   'matching_rsid_pmid10': matching_rsid_pmid10,
                   'matching_rsid_pmid100': matching_rsid_pmid100,
                   'matching_top10mesh':str(top10mesh),
                   'matching_top10rsid':str(top10rsid),
                   }

        df_report = pd.DataFrame(report, index=[gene]).transpose()

        # SLOW STEP!----------------------
        # generate fist report.csv

        #if os.path.isfile(directory+'/GRPMX_report.csv'):
        #    df_report_complete = pd.read_csv(directory+'/GRPMX_report.csv', index_col=0)#).set_index('Unnamed: 0')
        #    df_report_complete = pd.concat([df_report_complete, df_report], axis=1)
        #    df_report_complete.to_csv(directory+'/GRPMX_report.csv')
        #else:
        #    df_report.to_csv(directory+'/GRPMX_report.csv') # solo la prima volta

        # FAST ALT------------
        df_report_complete = pd.concat([df_report_complete, df_report], axis=1)

        time_omega = datetime.now()
        full_runtime = time_omega - time_alpha
        print(gene+'_runtime:', full_runtime, ' Genes processed:', genes.index(gene), 'on ', len(genes))
        total_seconds = full_runtime.total_seconds()

        # save checkpoint----------------------
        if genes.index(gene) > 1 and genes.index(gene) % 200 == 0:
            complete_df = complete_df.reindex(columns=['gene','rsid', 'pmids', 'mesh'])
            complete_df.to_csv(directory+'/grpmx_filtered_output.csv')
            df_report_complete.to_csv(directory+'/GRPMX_report.csv')
            print("saved checkpoint")
        else:
            pass

    else:
        print(gene+' not present in DataBase')
        pass

#directory = r'H:\Il mio Drive\GRPM db\GRPM proteincodinggenes\grpm_random_04'
# Save complete csv
complete_df = complete_df.reindex(columns=['gene','rsid', 'pmids', 'mesh'])
complete_df.to_csv(directory+'/grpmx_filtered_output.csv')

df_report_complete.to_csv(directory+'/GRPMX_report.csv')

# #Update gene values (remove previous gene entry)
GRPMX_report = pd.read_csv(directory+'/GRPMX_report.csv', index_col=0)
time_load_1 = datetime.now()
for gene in grpm_genes_list:
    if gene+'.1' in GRPMX_report.columns:
        GRPMX_report = GRPMX_report.drop(columns = gene)
        GRPMX_report = GRPMX_report.rename(columns={gene+'.1': gene})
        print(genes.index(gene))
    else:
        pass
time_load_2 = datetime.now()
print(time_load_2-time_load_1)
GRPMX_report.to_csv(directory+'/GRPMX_report.csv')

time_finish = datetime.now()
time_batch = time_finish - time_start
print('time batch:',time_batch)
print('runtime/gene:', time_batch/len(genes))

MT-ND1_runtime: 0:00:01.571460  Genes processed: 0 on  15519
MT-ND2_runtime: 0:00:01.018999  Genes processed: 1 on  15519
MT-CO1_runtime: 0:00:00.995291  Genes processed: 2 on  15519
MT-CO2_runtime: 0:00:00.920060  Genes processed: 3 on  15519
MT-ATP8_runtime: 0:00:00.901400  Genes processed: 4 on  15519
MT-ATP6_runtime: 0:00:00.812399  Genes processed: 5 on  15519
MT-CO3_runtime: 0:00:00.884877  Genes processed: 6 on  15519
MT-ND3_runtime: 0:00:00.908779  Genes processed: 7 on  15519
MT-ND4L_runtime: 0:00:00.964827  Genes processed: 8 on  15519
MT-ND4_runtime: 0:00:01.063931  Genes processed: 9 on  15519
MT-ND5_runtime: 0:00:01.053971  Genes processed: 10 on  15519
MT-ND6_runtime: 0:00:00.970962  Genes processed: 11 on  15519
MT-CYB_runtime: 0:00:00.926339  Genes processed: 12 on  15519
SNTG2_runtime: 0:00:00.871691  Genes processed: 13 on  15519
INTS3_runtime: 0:00:00.821823  Genes processed: 14 on  15519
KIR3DL2_runtime: 0:00:00.808524  Genes processed: 15 on  15519
KIR3DL3_runtime:

In [13]:
# save checkpoint
complete_df = complete_df.reindex(columns=['gene','rsid', 'pmids', 'mesh'])
complete_df.to_csv(directory+'/grpmx_filtered_output.csv')

df_report_complete.to_csv(directory+'/GRPMX_report.csv')

restart_from = len(df_report_complete.T)
print('partial job, genes in survey '+topic_tag+' report:', restart_from)

partial job, genes in survey cvd report: 8465


In [15]:
df_report_complete.T


,reference_mesh,starting_pmidmesh,starting_pmid,starting_mesh,starting_rsid,matching_pmidmesh,matching_pmids,matching_mesh,matching_rsid,dropped_rsid,matching_mesh_ratio,matching_pmids_ratio,matching_pmidmesh_ratio,matching_rsid_ratio,matching_rsid_pmid10,matching_rsid_pmid100,matching_top10mesh,matching_top10rsid
MT-ND1,528,3603,269,764,105,229,140,51,88,17,0.067,0.52,0.064,0.838,5,0,"['Mitochondria', 'Mitochondrial Diseases', 'Re...","['rs199476118', 'rs41460449', 'rs1599988', 'rs..."
MT-ND2,528,3958,292,795,123,269,156,58,98,25,0.073,0.534,0.068,0.797,6,0,"['Mitochondria', 'Mitochondrial Diseases', 'Re...","['rs199476118', 'rs41460449', 'rs28357984', 'r..."
MT-CO1,528,2308,163,558,97,175,96,43,84,13,0.077,0.589,0.076,0.866,2,0,"['Mitochondria', 'Reactive Oxygen Species', 'M...","['rs28357984', 'rs1599988', 'rs200165736', 'rs..."
MT-CO2,528,1330,94,422,67,100,63,27,60,7,0.064,0.67,0.075,0.896,1,0,"['Mitochondria', 'Mitochondrial Diseases', 'Re...","['rs2000975', 'rs28358884', 'rs200165736', 'rs..."
MT-ATP8,528,1802,127,518,77,135,87,31,64,13,0.06,0.685,0.075,0.831,3,0,"['Mitochondria', 'Mitochondrial Diseases', 'Re...","['rs199476133', 'rs2001031', 'rs2000975', 'rs2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FABP9,528,9,1,9,1,1,1,1,1,0,0.111,1.0,0.111,1.0,0,0,['Fatty Acid-Binding Proteins'],['rs1234288204']
FABP4,528,288,19,138,18,51,16,20,16,2,0.145,0.842,0.177,0.889,0,0,"['Fatty Acid-Binding Proteins', 'Obesity', 'Ad...","['rs1054135', 'rs77878271', 'rs10808846', 'rs2..."
MGAT4D,528,40,3,31,3,3,2,3,2,1,0.097,0.667,0.075,0.667,0,0,"['Cardiovascular Diseases', 'Cholesterol Ester...","['rs1241050', 'rs1594468']"
IMPA1,528,273,19,193,20,5,4,5,4,16,0.026,0.211,0.018,0.2,0,0,"['Autonomic Nervous System', 'Cell Proliferati...","['rs200237076', 'rs204774', 'rs204786', 'rs755..."


In [ ]:
df_read = pd.read_csv(directory+'/grpmx_filtered_output.csv', index_col=0)#.gene.drop_duplicates()
#df_read.to_clipboard()
print('genes matching:', df_read.gene.nunique())
print('mesh matching:', df_read.mesh.nunique())
print('apply threshold in Analizer Module')
df_read

# Check results

In [ ]:
# Visualize GRPMX_report.csv
GRPMX_report = pd.read_csv(directory+'/GRPMX_report.csv', index_col=0).transpose().reset_index().rename(columns={'index':'gene'})
GRPMX_report.gene.drop_duplicates().to_clipboard()
print(len(GRPMX_report.gene.drop_duplicates()))
GRPMX_report

In [ ]:

GRPMX_report[['reference_mesh', 'starting_pmidmesh', 'starting_pmid','starting_mesh','starting_rsid', 'matching_pmidmesh', 'matching_pmids', 'matching_mesh','matching_rsid', 'dropped_rsid', 'matching_rsid_pmid10','matching_rsid_pmid100']] = GRPMX_report[['reference_mesh', 'starting_pmidmesh', 'starting_pmid','starting_mesh','starting_rsid', 'matching_pmidmesh', 'matching_pmids', 'matching_mesh','matching_rsid', 'dropped_rsid','matching_rsid_pmid10','matching_rsid_pmid100']].astype(int)

GRPMX_report[['matching_mesh_ratio', 'matching_pmids_ratio','matching_pmidmesh_ratio', 'matching_rsid_ratio']] = GRPMX_report[['matching_mesh_ratio', 'matching_pmids_ratio','matching_pmidmesh_ratio','matching_rsid_ratio']].astype(float)

GRPMX_report_less = GRPMX_report[['matching_pmids','matching_pmids_ratio','matching_mesh','matching_rsid','matching_top10mesh','matching_top10rsid']]

GRPMX_report_sort = GRPMX_report.sort_values(by= 'matching_pmids',ascending=False)
GRPMX_report[['gene', 'matching_pmidmesh', 'matching_pmids',
'matching_mesh', 'matching_rsid', 'dropped_rsid', 'matching_mesh_ratio',
'matching_pmids_ratio', 'matching_pmidmesh_ratio',
'matching_rsid_ratio', 'matching_rsid_pmid10', 'matching_rsid_pmid100',
'matching_top10mesh', 'matching_top10rsid']].T

In [ ]:
# Matching PMIDs in Database
GRPMX_report_sort = GRPMX_report.sort_values(by= 'matching_pmids',ascending=False)
GRPMX_report_sort

x = GRPMX_report_sort.gene.iloc[:40]
y = GRPMX_report_sort['matching_pmids'].iloc[:40]
plt.figure(figsize=(5, 8))
plt.title('Matching PMIDs in Database', loc='center',pad=10)

plt.barh(x,y)
plt.gca().invert_yaxis()
plt.tick_params(axis='x', which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
#plt.xlabel('pmid count', position=(0.5, 1.08))
plt.ylabel('genes')
plt.xlabel('matching pmid', position=(0.5, 1.08))
ax = plt.gca()
ax.xaxis.set_label_position('top')
#plt.savefig(r'C:\Users\Public\Database_SSD\GRPX_heavyload (RunOnly)\logs\Database_Gene('+str(len(GRPMX_report.gene))+')-PMID_filtered.png',dpi=300, bbox_inches = "tight")
#plt.clf()

In [ ]:
# Add "interest index" to report:----------------------------------------------------------
max_match_pmids = int(GRPMX_report['matching_pmids'].max())
GRPMX_report_int = GRPMX_report
GRPMX_report_int['matching_pmids_index'] = round((GRPMX_report_int['matching_pmids']/max_match_pmids),3)

GRPMX_report_int['interest_index'] = round(GRPMX_report_int['matching_pmids_index'] * GRPMX_report_int['matching_pmids_ratio'],3)

GRPMX_report_int.set_index('gene').T.to_csv(directory+'/GRPMX_report_int.csv')

In [ ]:
plt.clf()# Matching PMIDs in Database
GRPMX_report_sort = GRPMX_report.sort_values(by= 'matching_pmids_index',ascending=False)

x = GRPMX_report_sort.gene.iloc[:100]
y = GRPMX_report_sort['matching_pmids_index'].iloc[:100]
plt.figure(figsize=(5, 20))
plt.title('Matching PMIDs in Database', loc='center',pad=10)

plt.barh(x,y)
plt.gca().invert_yaxis()
plt.tick_params(axis='x', which='both', top=True, bottom=False, labeltop=True, labelbottom=False)
#plt.xlabel('pmid count', position=(0.5, 1.08))
plt.ylabel('genes')
plt.xlabel('matching pmid', position=(0.5, 1.08))
ax = plt.gca()
ax.xaxis.set_label_position('top')
#plt.savefig(r'C:\Users\Public\Database_SSD\GRPX_heavyload (RunOnly)\logs\Database_Gene('+str(len(GRPMX_report.gene))+')-PMID_filtered.png',dpi=300, bbox_inches = "tight")
#plt.clf()

In [ ]:
import matplotlib.pyplot as plt

# Define the data
x = ['foo', 'bar', 'gal']
y = [10, 12, 14]

# Create a bar plot
plt.bar(x, y)

# Add labels and title
plt.xlabel('Category')
plt.ylabel('Count')
plt.title('Categorical Data')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create some sample data
data = np.random.normal(0, 1, 1000)

# Create an array of bin edges (in non-increasing order)
bins = np.array([0.0, 0.5, 1.0, 0.2, 2.0])

# Sort the bin edges in increasing order
bins = np.sort(bins)

# Create a histogram with the sorted bin edges
plt.hist(data, bins=bins)
plt.show()
